# Review

In [13]:
import numpy as np
import cv2
import sys
import os
import glob
import matplotlib.pyplot as plt

In [10]:
src = cv2.imread('./fig/puppy.bmp')
if src is None:
    print('image read failed')
    sys.exit()
    
M = np.array([[1,0.2,100],
             [0,1,100]], dtype=np.float32)
dst = cv2.warpAffine(src, M, (0,0))
dst_resize = cv2.resize(src, (0,0), fx=2, fy=1,
                       interpolation = cv2.INTER_LINEAR)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.imshow('resize', dst_resize)

cv2.waitKey()
cv2.destroyAllWindows()

In [12]:
src = cv2.imread('./fig/puppy.bmp')
if src is None:
    print('image read failed')
    sys.exit()
print(src.shape)
h, w = src.shape[:2]
cp = (w/2, h/2)
M = cv2.getRotationMatrix2D(cp,30,1)

dst = cv2.warpAffine(src, M, (0,0))

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

(480, 640, 3)


In [14]:
src = cv2.imread('./fig/checkerboard.png')

h, w = src.shape[:2]

src_point = np.array([[217,50], [691,47], [830,517], [67,526]],np.float32)
dst_point = np.array([[0,0], [w-1,0],[w-1,h-1],[0,h-1]],np.float32)

M = cv2.getPerspectiveTransform(src_point, dst_point)
dst = cv2.warpPerspective(src, M, (w,h))

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [20]:
src = cv2.imread('./fig/blue_eyes.png', cv2.IMREAD_REDUCED_COLOR_2)

# kernel_3 = np.ones((3,3), np.float32)/9
# kernel_5 = np.ones((5,5), np.float32)/25

# dst3 = cv2.filter2D(src, -1, kernel_3)
# dst5 = cv2.filter2D(src, -1, kernel_5)

dst3 = cv2.blur(src, (3,3))
dst5 = cv2.blur(src, (5,5))

dst_gaussian = cv2.GaussianBlur(src, (0,0), 1)
dst_bilateral = cv2.bilateralFilter(src, -1, 1,1)

cv2.imshow('src', src)
cv2.imshow('dst3', dst3)
cv2.imshow('dst5', dst5)
cv2.imshow('dst_gaussian', dst_gaussian)
cv2.imshow('dst_bilateral', dst_bilateral)

cv2.waitKey()
cv2.destroyAllWindows()

In [24]:
src = cv2.imread('./fig/spring_in_park.jpg', cv2.IMREAD_REDUCED_COLOR_2)
if src is None:
    print('image read failed')
    sys.exit()
src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(src_hsv)

v_equal = cv2.equalizeHist(v)
v_norm = cv2.normalize(v, None, 0,255, cv2.NORM_MINMAX)

src_equal = cv2.merge((h,s,v_equal))
src_norm = cv2.merge((h,s,v_norm))

equal = cv2.cvtColor(src_equal, cv2.COLOR_HSV2BGR)
norm = cv2.cvtColor(src_norm, cv2.COLOR_HSV2BGR)

cv2.imshow('src', src)
cv2.imshow('equal', equal)
cv2.imshow('norm', norm)

cv2.waitKey()
cv2.destroyAllWindows()

In [25]:
def call_track(pos):
    hmin = cv2.getTrackbarPos('min','src')
    hmax = cv2.getTrackbarPos('max','src')
    dst = cv2.inRange(src_hsv, (hmin,150,0), (hmax,255,255))
    cv2.imshow('src', dst)

src = cv2.imread('./fig/palette_round.jpg')
src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

cv2.imshow('src', src)
cv2.createTrackbar('min', 'src', 0, 179, call_track)
cv2.createTrackbar('max', 'src', 1, 179, call_track)

cv2.waitKey()
cv2.destroyAllWindows()

In [27]:
src = cv2.imread('./fig/green.png', cv2.IMREAD_REDUCED_COLOR_2)

src_ycrcb = cv2.cvtColor(src, cv2.COLOR_BGR2YCR_CB)

x,y,w,h = cv2.selectROI(src)
crop = src_ycrcb[y:y+h, x:x+w]

channels = [1,2]
ranges = [0,256, 0,256]
hist = cv2.calcHist([crop],channels,None,[256,256],ranges)
backproj = cv2.calcBackProject([src_ycrcb],channels,hist,ranges,1)
dst = cv2.copyTo(src, backproj)

cv2.imshow('src', src)
cv2.imshow('backproj', backproj)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [2]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Video open failed')
    sys.exit()
    
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)*0.7)
fourcc = cv2.VideoWriter_fourcc(*"DIVX")

out = cv2.VideoWriter('output.avi', fourcc, fps, (w,h))

while True:
    ret, frame = cap.read()
    if not ret:
        print("Video read failed")
        break
        
    cv2.imshow('frame', frame)
    out.write(frame)
    
    kw = cv2.waitKey(30)
    if kw==27:
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()

In [4]:
def call_mouse(event, x,y, flags, params):
    global oldx, oldy
    if event==cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x,y
    elif event==cv2.EVENT_MOUSEMOVE:
        if flags&cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx,oldy), (x,y), (0,0,0), 3, cv2.LINE_AA)
            cv2.imshow('img', img)
            oldx,oldy = x,y
            
img = np.ones((500,600,3), np.uint8)*255

cv2.imshow('img', img)
cv2.setMouseCallback('img', call_mouse)

cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
img = cv2.imread('./fig/puppy.bmp')
if img is None:
    print('image read failed')
    sys.exit()
    
cv2.namedWindow('image', cv2.WINDOW_AUTOSIZE)
cv2.imshow('image', img)

while True:
    kw = cv2.waitKey()
    if kw==27 or kw==ord('q'):
        break
        
cv2.destroyAllWindows()

# DNN
## image classifier

In [16]:
filename = './googlenet/fig/beagle.jpg'
filenames = os.listdir('./googlenet/fig/')
# print(filenames

img = cv2.imread(filename)
if img is None:
    print('image read failed')
    sys.exit()
print(img.shape)

model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet/deploy.prototxt'

net = cv2.dnn.readNet(model, config)
if net.empty():
    print('Network load failed')
    sys.exit()
print('Network load complete')

classNames = []
classfile = './googlenet/classification_classes_ILSVRC2012.txt'
with open(classfile, 'r') as f:
    classNames = f.read().rstrip('\n').split('\n')
print(len(classNames))

blob = cv2.dnn.blobFromImage(img, 1, (244,244), (104,117,12),
                            swapRB = False)
net.setInput(blob)
prob = net.forward()
# print(prob.shape)

out = prob.flatten()
# print(out.shape)
classId = np.argmax(out)
confidence = out[classId]
category = classNames[classId]
text = f'{category} ({confidence*100:4.2f} %)'
cv2.putText(img, text, (10,30), cv2.FONT_HERSHEY_COMPLEX,
           1, (0,0,255), 1, cv2.LINE_AA)

# print(classId)
# print(classNames[classId])
# print(out[classId])

cv2.imshow('img',img)
cv2.waitKey()
cv2.destroyAllWindows()

['apple1.png', 'apple2.png', 'beagle.jpg', 'fish.jpg', 'pineapple.jpg', 'scooter.jpg', 'space_shuttle.jpg']
(366, 640, 3)
Network load complete
1000


In [33]:
imgs = os.listdir('./googlenet/fig/')
# print(filenames)
img_paths=[]
for img in imgs:
    path = './googlenet/fig/' + img
    img_paths.append(path)
# print(len(img_paths))
# print(img_paths)

model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet/deploy.prototxt'

net = cv2.dnn.readNet(model, config)
if net.empty():
    print('Network load failed')
    sys.exit()
print('Network load complete')

classNames = []
classfile = './googlenet/classification_classes_ILSVRC2012.txt'
with open(classfile, 'r') as f:
    classNames = f.read().rstrip('\n').split('\n')
print(len(classNames))

cv2.namedWindow('img', cv2.WINDOW_AUTOSIZE)

idx=0
while True:
    img = cv2.imread(img_paths[idx])
    if img is None:
        print('image read failed')
        sys.exit()
#     print(img.shape)
    img = cv2.resize(img, (500,500))
    
    blob = cv2.dnn.blobFromImage(img, 1, (244,244), (104,117,12),
                            swapRB = False)
    net.setInput(blob)
    prob = net.forward()
# print(prob.shape)

    out = prob.flatten()
# print(out.shape)
    classId = np.argmax(out)
    confidence = out[classId]
    category = classNames[classId]
    text = f'{category} ({confidence*100:4.2f} %)'
    cv2.putText(img, text, (10,490), cv2.FONT_HERSHEY_COMPLEX,
               1, (0,0,255), 1, cv2.LINE_AA)

    cv2.imshow('img',img)
    if cv2.waitKey(1500)==27:
        break
        
    idx +=1
    if idx >= len(img_paths):
        idx=0

cv2.destroyAllWindows()

Network load complete
1000


## face detect

In [39]:
img = cv2.imread('./fig/sunglass.png')

model = './face_detection/opencv_face_detector_uint8.pb'
config = './face_detection/opencv_face_detector.pbtxt'

face_detect_net = cv2.dnn.readNet(model, config)
if face_detect_net.empty():
    print('Network load error')
    sys.exit()
    
blob = cv2.dnn.blobFromImage(img, 1, (300,300), (104,177,123),
                            swapRB=False)
face_detect_net.setInput(blob)
out = face_detect_net.forward()
detect = out[0,0,:,:]
h,w = img.shape[:2]
for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    
    if confidence > 0.5:
        x1 = int(detect[i, 3]*w)
        y1 = int(detect[i, 4]*h)
        x2 = int(detect[i, 5]*w)
        y2 = int(detect[i, 6]*h)
        
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255), 2)
        text = 'Face: {:.2f}%'.format(confidence*100)
        
        cv2.putText(img,text, (x1,y1-2), cv2.FONT_HERSHEY_COMPLEX,
                   0.8, (0,0,255), 1, cv2.LINE_AA)
        
    
cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [43]:
model = './face_detection/opencv_face_detector_uint8.pb'
config = './face_detection/opencv_face_detector.pbtxt'

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cap opened failed")
    sys.exit()
    
net = cv2.dnn.readNet(model, config)
if net.empty():
    print("Network load failed")
    sys.exit()
    
while True:
    ret, frame = cap.read()
    if not ret:
        print("frame read failed")
        break
        
    blob = cv2.dnn.blobFromImage(frame, 1, (300,300), (104,177,123))
    net.setInput(blob)
    out = net.forward()
    detect = out[0,0,:,:]
    h,w = frame.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i,2]
        if confidence >0.5:
            x1 = int(detect[i, 3]*w)
            y1 = int(detect[i, 4]*h)
            x2 = int(detect[i, 5]*w)
            y2 = int(detect[i, 6]*h)

            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,255), 2)
            text = 'Face: {:.2f}%'.format(confidence*100)

            cv2.putText(frame, text, (x1,y1-2), cv2.FONT_HERSHEY_COMPLEX,
                       0.8, (0,0,255), 1, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    if cv2.waitKey(30)==27:
        break
        
cap.release()
cv2.destroyAllWindows()

In [42]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('open failed')
    sys.exit()

## MNIST

In [ ]:
from tensorflow import keras

In [ ]:
(x_train, y_train), (x_test,y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1)/255.
x_test = x_test.reshape(-1,28,28,1)/255.
print(x_train.shape, x_train.dtype)
print(x_test.shape, x_test.dtype)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3,3),
                             input_shape=(28,28,1),
                             activation='relu'))
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation=('relu')))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics='accuracy')

In [ ]:
modelpath = './mnist_my_model/{epoch:002d}-{val_loss:.4f}.h5'
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=modelpath,
    save_best_only=True,
    verbose=True
)
stoppoint = keras.callbacks.EarlyStopping(patience=10)

hist = model.fit(x_train, y_train, epochs=30, batch_size=200,
                verbose=1, validation_split=0.3,
                 callbacks=[checkpoint, stoppoint])

In [ ]:
print(model.evaluate(x_test, y_test))

In [ ]:
model.save('./mnist_onnx/', include_optimizer=False)

In [54]:
# ! pip install -U tf2onnx

In [ ]:
!python -m tf2onnx.convert --saved-model mnist_onnx --output model_mnist.onnx

In [14]:
def norm_digit(img):
    # 무게 중심 좌표 추출
    m = cv2.moments(img)
    cx = m['m10'] / m['m00']
    cy = m['m01'] / m['m00']
    h, w = img.shape[:2]
    
    # affine 행렬 생성
    aff = np.array([[1, 0, w/2 - (cx + 0.5)], [0, 1, h/2 - (cy + 0.5)]], 
                   dtype=np.float32)
    
    # warpAffine을 이용해 기하학 변환
    dst = cv2.warpAffine(img, aff, (0, 0))
    return dst

def on_mouse(event, x,y, flags, params):
    global oldx, oldy
    if event==cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x,y
    elif event==cv2.EVENT_MOUSEMOVE:
        if flags&cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx,oldy), (x,y), 255, 20, cv2.LINE_AA)
            cv2.imshow('image', img)
            oldx,oldy = x,y
            
            
net = cv2.dnn.readNet('./model_mnist.onnx')
if net.empty():
    print('Network load failed')
    sys.exit()

img = np.zeros((400,400), np.uint8)
cv2.imshow('image', img)
cv2.setMouseCallback('image', on_mouse)

while True:
    key = cv2.waitKey()
    if key==27:
        break
    elif key == ord(' '):
        blob= cv2.dnn.blobFromImage(norm_digit(img), 1/255,(28,28))
        net.setInput(blob)
        prob = net.forward()
        
        _, maxVal, _, maxLoc = cv2.minMaxLoc(prob)
        digit = maxLoc[0]
        print(f'{digit} ({maxVal*100:4.2f}%)')
        img.fill(0)
        cv2.imshow('image',img)
    
cv2.destroyAllWindows()

2 (97.37%)
1 (99.93%)
3 (59.62%)
3 (88.62%)
3 (52.03%)
9 (67.59%)
7 (99.86%)
5 (48.86%)
3 (48.03%)
3 (45.53%)
9 (45.85%)


In [11]:
cv2.destroyAllWindows()